# Integración de Jupyter y repositorios Git. Publicación de notebooks

![github_logo](img/github_logo.png)

+ Un fichero notebook serializa el contenido de cada celda en formato JSON...
+ ...y además contiene metadatos, texto formateado, datos enriquecidos...
+ Dificultad a la hora de controlar versiones.
+ La salida de cada celda (imágenes, gráficas, tablas con datos) se puede almacenar junto con el notebook.
    - Esto **facilita** la tarea a la hora de compartir notebooks con los resultados visibles para que el receptor no tenga que ejecutar nada:
      + Evita problemas de librerías y dependencias.
      + Ahorra tiempo.
    - Pero esto **dificulta** el almacenamiento de los notebooks en sistemas de control de versiones como Git:
      + Si se hace un cambio en una celda de código probablemente también cambie el resultado a mostrar.
      + Complica los `diffs`.


+ Cuando se habla de compartir notebooks, normalmente se hace referencia a uno de los dos siguientes paradigmas:
  - El notebook muestra resultados finales (como este notebook).
    + Se puede guardar el resultado de las celdas.
    + Fácil de compartir, fácil de visualizar y de entender.
  - El notebook se entiende como un medio interactivo con el que experimentar.
    + Se puede eliminar la salida de las celdas del notebook.
    + Fácil de modificar, fácil de extender.

### Problemática de Jupyter + Git: Conflictos

+ Se interactúa con un notebook en un repositorio Git.
+ Surge un conflicto debido a diversos motivos:
  - Dos desarrolladores lo editan simultáneamente e intentan guardar sus cambios.
  - Un usuario ejecuta la hoja y guarda la hoja con los resultados.
  - Un usuario abre la hoja teniendo una versión distinta de Jupyter (los metadatos cambian).
  - Los resultados cambian (por ejemplo porque hay números aleatorios involucrados).

+ ...al hacer `push` surge un conflicto y la hoja queda inutilizable:

![git_error_1](img/git_error_1.png)

+ El notebook se almacena en formato JSON, pero Git guarda el conflicto en texto plano:

![git_error_2](img/git_error_2.png)

+ Esto no es un formato JSON válido, y Jupyter arrojará errores al intentar abrir la hoja.
+ ¿Solución? Editar el notebook **a mano** desde un editor de texto.
+ Posible solución: **nbdev2**:
  - https://nbdev.fast.ai/
+ nbdev2 proporciona una serie de _hooks_ a algunas operaciones en Git:
  - Encapsula los mensajes de conflictos **como celdas** para que Jupyter pueda interpretarlos correctamente.
  - Proporciona _hooks_ para las operaciones **merge** y **save**.
  - ...entre otras cosas.
+ Con esta herramienta Jupyter puede abrir notebooks con conflictos.
+ Para resolver el conflicto, el usuario simplemente **elimina** las celdas que muestran los resultados que no se quieren mantener, o bien **combina** las celdas adecuadamente.

![git_error_3](img/git_error_3.png)

## Almacenando notebooks en GitHub

+ El notebook se almacenará con exactamente la misma apariencia que tenía cuando se guardó por última vez.

### Solución 1: Limpiar la salida manualmente

+ La solución **más sencilla** para asegurar que el notebook está listo para ser compartido:
  - `Cell->All Output->Clear`.
+ Si además se desea ejecutar el notebook y guardar las celdas de salida:
  - `Kernel->Restart & Run All`
+ Esperar a que se ejecuten todas las celdas y entonces:
  - `File->Save and Checkpoint`
  - `File->Close and Halt`
+ Esto asegura que el notebook no se queda en un estado intermedio o a media ejecución.
+ Nota extra: Añadir el directorio **.ipynb_checkpoints** al fichero **.gitignore**.
+ **Problemas**: Proceso manual, se requiere volver a ejecutar el notebook, otros colaboradores pueden meter más metadatos.


+ Idea: Trabajar con dos ramas en el repositorio:
  - `dev branch`: Los editores del notebook trabajan sobre esta rama. No se almacena la información de las celdas de salida.
  - `master branch`: Contiene un `commit` adicional en el que los notebooks han sido ejecutados y la salida de las celdas guardada.

### Solución 2: Utilizar herramientas externas como ReviewNB

https://www.reviewnb.com/  
https://blog.reviewnb.com/github-jupyter-notebook/

+ Aplicación GitHub que ofrece un `diff` visual para capturar la diferencia entre notebooks de manera sencilla.
+ Los `diff` del JSON del notebook se vuelven confusos y poco prácticos si hay datos binarios en el notebook.
+ El software está pensado específicamente para su integración con GitHub: Poco flexible, pero fácil de manejar.

![reviewnb_1](img/reviewnb_1.png)

![reviewnb_2](img/reviewnb_2.png)

![reviewnb_0](img/reviewnb_0.png)

### Bonus: Extensión de Git en Jupyter Lab

+ Jupyter tiene disponible una extensión **Git** para la interfaz Lab.
+ Interfaz point and click.

![jupyter_git](img/jupyter_git.png)

+ Mediante la pestaña correspondiente (a la izquierda) se pueden añadir ficheros al stage.
+ También se pueden controlar los cambios realizados a los ficheros.
+ Se pueden hacer commits desde la misma interfaz. Similar a otros IDEs.

![jupyter_gitinit](img/jupyter_gitinit.png)

## Publicación de notebooks

### Mecanismos nativos

+ Jupyter proporciona mecanismos nativos para exportar notebooks directamente en formato HTML o PDF:
  - `File->Download As->...`
+ Similar a usar el comando `jupyter nbconvert --to <format> <file>` desde la línea de comandos.
+ https://jupyterlab.readthedocs.io/en/stable/user/export.html
+ https://nbconvert.readthedocs.io/en/latest/usage.html
+ Esto exporta el notebook tal cual, así como los resultados generados.
+ Útil para compartir texto, demostraciones, resultados de investigación y ejemplos no interactivos.
+ **Desventaja**: No permite la ejecución del notebook.

### Visualización en GitHub

+ GitHub soporta actualmente el renderizado de ficheros _.ipynb_ directamente.
+ El notebook se muestra como una página web estática.
+ No es posible ejecutar celdas o modificar el notebook.

![github_shownotebook](img/github_shownotebook.png)

+ **Alternativa**: Uso de Gist (https://gist.github.com/).
+ El servicio **Gist** se utiliza para compartir snippets de código:
  - Se copia el contenido JSON del notebook (fichero _.ipynb_) en el gist.
  - Se da un nombre al fichero gist y la extensión _.ipynb_.
  - `Create public gist`.
  - El servicio proporciona una URL pública que se puede compartir para visualizar el notebook.

![gist_screen](img/gist_screen.png)

### Visualización con nbviewer

https://nbviewer.org/

+ Nbviewer forma parte del proyecto Jupyter, y permite también mostrar notebooks almacenados en un repositorio público.
  - Git, Dropbox, Google Drive...

![nbviewer_load](img/nbviewer_load.png)

+ Este proyecto fue desarrollado antes de que GitHub integrara la renderización de hojas Jupyter.
+ Recibe como entrada una URL, un ID Gist o un repositorio GitHub.
+ Renderiza un notebook como **página web**.
+ Proporciona una **URL para compartir el resultado**.

![nbviewer_show](img/nbviewer_show.png)

+ **Ojo**: Nbviewer no permite la edición o ejecución del notebook.

### Uso de Binder

[https://mybinder.org/](https://mybinder.org/)

+ **MyBinder** (o **Binder**) permite crear proyectos a partir de repositorios Git y proporciona **entornos de ejecución** para los notebooks almacenados.


+ **Requisito**: El repositorio Git debe ser **público**.
  - Binder no proporciona gestión de tokens de autentificación.
  - Es suficiente con usar un repositorio público GitHub.
  - Repositorio de ejemplo: `https://github.com/dsevilla/data-science-ipython-notebooks`
  - El Binder también será público:
    + Nada de escribir contraseñas ni pasar tokens de autentificación.


+ **Funcionamiento**:

![binder_howitworks](img/binder_howitworks.png)

+ **Paso 1**: Acceder a https://mybinder.org y rellenar los campos con la información adecuadamente:

![binder_screen](img/binder_screen.png)

+ **Paso 2**: Acceder al link que proporciona Binder mientras se crea el entorno. El link será del estilo a: `https://mybinder.org/v2/gh/YOUR-USERNAME/my-first-binder/HEAD`. Binder está mientras tanto efectuando los siguientes pasos:
  + Buscando el repositorio GitHub proporcionado.
  + Analizando los contenidos del repositorio.
  + Creando una imagen Docker basada en el repositorio.
  + Lanzando la imagen Docker en la nube.
  + Conectando al usuario a la imagen mediante el navegador.

![binder_load](img/binder_load.png)

+ **Paso 3**: Una vez la imagen se ha lanzado y tenemos acceso, veremos una interfaz similar a la de Jupyter Lab y tendremos acceso para ejecutar cualquier notebook del repositorio:

![binder_show](img/binder_show.png)

+ **(Opcional) Paso 4**: Para añadir dependencias es necesario crear un fichero `requirements.txt`.
  + Una línea por dependencia.
  + `numpy==1.14.5`: Hace que Binder descargue la librería numpy con versión igual a la indicada.
  + Las versiones de las dependencias también se pueden indicar con los símbolos `>=`, o no indicarse para traer la dependencia más actual posible.
  + Ahora, durante el lanzamiento en el paso 2, se introduce un paso adicional en el que Binder descarga las dependencias.


+ **(Opcional) Paso 5**: Añadir ficheros de datos mediante un fichero `postBuild`.
  + Los ficheros de datos pequeños se pueden añadir directamente al repositorio GitHub y serán accesibles desde Binder (menos de 10MB).
  + Los ficheros de datos de mayor tamaño se pueden incluir en el Binder mediante el fichero `postBuild`.
  + Un fichero `postBuild` es un script que se ejecuta una única vez cuando se construye la imagen por primera vez.
    - Se puede usar para, mediante instrucciones `wget` por ejemplo, incluir ficheros de datos en la imagen Docker.

+ **Paso 6**: Compartir la URL dada en un principio. Binder creará un entorno en el que el usuario puede navegar por el repositorio, abrir notebooks, modificarlos y ejecutarlos.

### Google Colab

![colab_intro](img/colab_intro.png)

https://colab.research.google.com/

+ Un notebook también se puede compartir utilizando **Google Colab**.
+ Veremos los detalles del entorno Google Colab en la **siguiente sesión**.
+ Google Colab proporciona una serie de notebooks de ejemplo sobre distintos apartados y funcionalidad:
  - **Overview of Colaboratory Features**
  - **Markdown Guide**
  - **Charts in Colaboratory**
  - **External data: Drive, Sheets, and Cloud Storage**
  - ...


+ Google Colab permite importar notebooks almacenados en **otros medios**:
  - `Archivo->Abrir/Subir cuaderno`
  - En Google Drive
  - En GitHub
    + **OJO**: Si se desea abrir un repositorio privado es necesario dar a Colab permisos adicionales.
  - Ficheros locales


+ Hace uso de **Google Cloud Platform** para proporcionar un entorno de ejecución a los notebooks importados.

![colab_open](img/colab_open.png)

+ Una vez que se ha abierto un notebook, este se puede almacenar localmente en **Google Drive** o **GitHub**.
  - Creación de una carpeta llamada **Colab Notebooks** en el Google Drive del usuario.


+ El notebook ahora puede modificarse y ejecutarse.

![colab_notebook](img/colab_notebook.png)